<a href="https://colab.research.google.com/github/gideaoaugustom/ETL_Python_Pandas_Involves/blob/main/ETL_Python_Pandas_Involves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import os

# Lê o dataset com codificação corrigida
df = pd.read_csv('/content/dataset_teste_de.csv', sep=';', encoding='latin1')

print("Dataset carregado com sucesso !")
print(df.shape)
df.head()


Dataset carregado com sucesso !
(40, 10)


,ID_DATASET,ID_PONTO_VENDA,NOME_PONTO_VENDA,PERFIL_PONTO_VENDA,DATA,ID_LINHA_PRODUTO,NOME_LINHA_PRODUTO,MARCA_LINHA_PRODUTO,TIPO_COLETA,VALOR
0,1021,115,INVOLVES,ATACADAO,01/09/2020,398,BISCOITOS SORTIDOS,INVOLVES ZERO,DISPONIBILIDADE,SIM
1,1022,115,INVOLVES,ATACADAO,02/09/2020,407,MARGARINA,INVOLVES ZERO,DISPONIBILIDADE,SIM
2,1023,115,INVOLVES,ATACADAO,03/09/2020,408,MANTEIGA,INVOLVES ZERO,DISPONIBILIDADE,SIM
3,1024,115,INVOLVES,ATACADAO,04/09/2020,422,LEITE EM PO,INVOLVES ZERO,DISPONIBILIDADE,NÃO
4,1025,115,INVOLVES,ATACADAO,05/09/2020,423,MAISENA,INVOLVES ZERO,DISPONIBILIDADE,SIM


In [41]:
# --- Dimensão Calendário ---
dim_calendario = df[['DATA']].drop_duplicates().copy()
dim_calendario['DATA'] = pd.to_datetime(dim_calendario['DATA'], errors='coerce')
dim_calendario['MES'] = dim_calendario['DATA'].dt.month
dim_calendario['ANO'] = dim_calendario['DATA'].dt.year
dim_calendario['ID_CALENDARIO'] = range(1, len(dim_calendario)+1)

# --- Dimensão Ponto de Venda ---
dim_pdv = df[['ID_DATASET', 'NOME_PONTO_VENDA', 'PERFIL_PONTO_VENDA']].drop_duplicates().copy()

# --- Dimensão Linha de Produto ---
dim_linha = df[['ID_LINHA_PRODUTO', 'NOME_LINHA_PRODUTO', 'MARCA_LINHA_PRODUTO']].drop_duplicates().copy()

print("Tabelas dimensionais criadas !")




Tabelas dimensionais criadas !


In [42]:
# Filtra registros de Disponibilidade com VALOR = SIM
disp = df[(df['TIPO_COLETA'] == 'Disponibilidade') & (df['VALOR'].str.upper() == 'SIM')].copy()

# Fato Disponibilidade: contagem de presenças por PDV e linha de produto
ft_disp = (disp.groupby(['ID_PONTO_VENDA', 'ID_LINHA_PRODUTO'])
                .size()
                .reset_index(name='QTD_PRESENCA'))

# Adiciona chaves de data (referência a setembro/2020)
ft_disp = ft_disp.merge(dim_calendario, left_on=None, right_on=None, how='cross')
ft_disp = ft_disp[ft_disp['MES'] == 9]
ft_disp = ft_disp[['ID_PONTO_VENDA', 'ID_LINHA_PRODUTO', 'ID_CALENDARIO', 'QTD_PRESENCA']]

# Fato Disponibilidade Agregada: total por PDV
ft_disp_agr = (ft_disp.groupby(['ID_PONTO_VENDA', 'ID_CALENDARIO'])['QTD_PRESENCA']
                      .sum()
                      .reset_index(name='QTD_PRESENCA_TOTAL'))

print("Fatos de Disponibilidade criados com sucesso!")


Fatos de Disponibilidade criados com sucesso!


In [43]:
# Filtra registros de Ponto Extra
ponto = df[df['TIPO_COLETA'] == 'Ponto Extra'].copy()

# Fato Ponto Extra: soma de registros por PDV e linha de produto
ft_ponto = (ponto.groupby(['ID_PONTO_VENDA', 'ID_LINHA_PRODUTO'])
                   .size()
                   .reset_index(name='QTD_PONTO_EXTRA'))

# Vincula com calendário de setembro/2020
ft_ponto = ft_ponto.merge(dim_calendario, left_on=None, right_on=None, how='cross')
ft_ponto = ft_ponto[ft_ponto['MES'] == 9]
ft_ponto = ft_ponto[['ID_PONTO_VENDA', 'ID_LINHA_PRODUTO', 'ID_CALENDARIO', 'QTD_PONTO_EXTRA']]

# Fato Ponto Extra Agregada: soma total por PDV e calendário
ft_ponto_agr = (ft_ponto.groupby(['ID_PONTO_VENDA', 'ID_CALENDARIO'])['QTD_PONTO_EXTRA']
                         .sum()
                         .reset_index(name='QTD_PONTO_EXTRA_TOTAL'))

print("Fatos de Ponto Extra criados com sucesso!")


Fatos de Ponto Extra criados com sucesso!


In [44]:
output_path = '/content/output'
os.makedirs(output_path, exist_ok=True)

# Salva todas as tabelas como CSV
dim_calendario.to_csv(f'{output_path}/DIM_CALENDARIO.csv', index=False)
dim_pdv.to_csv(f'{output_path}/DIM_PONTO_VENDA.csv', index=False)
dim_linha.to_csv(f'{output_path}/DIM_LINHA_PRODUTO.csv', index=False)
ft_disp.to_csv(f'{output_path}/FT_DISPONIBILIDADE.csv', index=False)
ft_disp_agr.to_csv(f'{output_path}/FT_DISPONIBILIDADE_AGREGADA.csv', index=False)
ft_ponto.to_csv(f'{output_path}/FT_PONTO_EXTRA.csv', index=False)
ft_ponto_agr.to_csv(f'{output_path}/FT_PONTO_EXTRA_AGREGADA.csv', index=False)

print("Arquivos CSV gerados em /content/output com sucesso!")
print(os.listdir(output_path))


Arquivos CSV gerados em /content/output com sucesso!
['FT_DISPONIBILIDADE_AGREGADA.csv', 'DIM_LINHA_PRODUTO.csv', 'DIM_CALENDARIO.csv', 'DIM_PONTO_VENDA.csv', 'FT_PONTO_EXTRA.csv', 'FT_DISPONIBILIDADE.csv', 'FT_PONTO_EXTRA_AGREGADA.csv']


In [45]:
print("DIM_LINHA_PRODUTO:")
print(dim_linha.head())

print("\nFT_DISPONIBILIDADE_AGREGADA:")
print(ft_disp_agr.head())

print("\nFT_PONTO_EXTRA_AGREGADA:")
print(ft_ponto_agr.head())


DIM_LINHA_PRODUTO:
   ID_LINHA_PRODUTO  NOME_LINHA_PRODUTO MARCA_LINHA_PRODUTO
0               398  BISCOITOS SORTIDOS       INVOLVES ZERO
1               407           MARGARINA       INVOLVES ZERO
2               408            MANTEIGA       INVOLVES ZERO
3               422         LEITE EM PO       INVOLVES ZERO
4               423             MAISENA       INVOLVES ZERO

FT_DISPONIBILIDADE_AGREGADA:
Empty DataFrame
Columns: [ID_PONTO_VENDA, ID_CALENDARIO, QTD_PRESENCA_TOTAL]
Index: []

FT_PONTO_EXTRA_AGREGADA:
Empty DataFrame
Columns: [ID_PONTO_VENDA, ID_CALENDARIO, QTD_PONTO_EXTRA_TOTAL]
Index: []


In [46]:
print("Processo total finalizado com sucesso!")

Processo total finalizado com sucesso!
